# 1. 디스크립터 예: 속성 검증

## 1.1 LineItem 버전 #3: 간단한 디스크립터

In [66]:
class Quantity:  # <1>

    def __init__(self, storage_name):
        self.storage_name = storage_name  # <2>

    def __set__(self, instance, value):  # <3>
        if value > 0:
            instance.__dict__[self.storage_name] = value  # <4>
        else:
            raise ValueError('value must be > 0')

In [67]:
class LineItem:
    weight = Quantity('weight')  # <5>
    price = Quantity('price')  # <6>

    def __init__(self, description, weight, price):  # <7>
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [68]:
truffle = LineItem('White truffle', 100, 2)

In [69]:
dir(truffle)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'description',
 'price',
 'subtotal',
 'weight']

In [71]:
truffle.weight

100

In [ ]:
# 저거 실행 시점이라고 해야 하나????? 생성되는 시점..? cv라 자바처럼 LineItem 클래스가 로딩될 때? cv라서 또 공통일 거 아님?

In [63]:
weightQuantity = LineItem.weight

In [64]:
weightQuantity.storage_name

'weight'

In [65]:
dir(weightQuantity)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'storage_name']

## 1.2 LineItem 버전 #4: 자동 저장소 속성명

In [99]:
class Quantity:
    __counter = 0  # <1>

    def __init__(self):
        cls = self.__class__  # <2>
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)  # <3>
        cls.__counter += 1  # <4>

    def __get__(self, instance, owner):  # <5>
        print('__get__ 호출')
        return getattr(instance, self.storage_name)  # <6>

    def __set__(self, instance, value):
        print('__set__ 호출')
        if value > 0:
            setattr(instance, self.storage_name, value)  # <7>
        else:
            raise ValueError('value must be > 0')

In [100]:
class LineItem:
    weight = Quantity()  # <8>
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [135]:
LineItem.weight

__get__ 호출


AttributeError: 'NoneType' object has no attribute '_Quantity#0'

In [101]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

__set__ 호출
__set__ 호출


In [102]:
coconuts.weight # __get__ 주석처리 시

__get__ 호출


20

In [103]:
test = coconuts.weight

__get__ 호출


In [104]:
test.storage_name

AttributeError: 'int' object has no attribute 'storage_name'

In [105]:
coconuts.weight

__get__ 호출


20

In [106]:
getattr(coconuts, '_Quantity#0')

20

In [107]:
dir(coconuts)

['_Quantity#0',
 '_Quantity#1',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'description',
 'price',
 'subtotal',
 'weight']

In [111]:
coconuts.__dict__ # 헐.. self.weight, price는 어디감 ㅠ 이거 앞에 복습을 해야 되는 거임? 속성 가려지는 거 어쩌구???? 아;;;

{'description': 'Brazilian coconut', '_Quantity#0': 20, '_Quantity#1': 17.95}

In [109]:
coconuts.__dict__['weight']

KeyError: 'weight'

In [ ]:
# 그러니까 지금 위의 상황은 디스크립터가 객체 속성을 가리는 거 같은데..?

In [112]:
class Class:
    data = 'the class data attr'
    
    @property
    def prop(self):
        return 'the prop value'

In [113]:
obj = Class()

In [114]:
obj.__dict__

{}

In [115]:
obj.data

'the class data attr'

In [116]:
obj.data = 'bar'

In [117]:
obj.__dict__

{'data': 'bar'}

In [118]:
obj.data

'bar'

In [119]:
Class.data

'the class data attr'

In [131]:
# 위 예제랑 똑같은 상황 만들려면 어케해야되지,, 지금 prop이라는 이름의 프로퍼티가 있는데, __init__에 prop이라는 거 만들면 되는 거 아님?
# !!! 프로퍼티 속성이 객체 속성보다 짱쎔 self.__dict__['prop']에 추가했는데 프로퍼티가 지금 얘 가리고 있다.. 쟤는 self.__dict__['prop'] 으로만 접근 가능하고 수정 가능!

In [127]:
class Class:
    data = 'the class data attr'
    
    def __init__(self, name):
        self.__dict__['prop'] = name
    
    @property
    def prop(self):
        return 'the prop value'

In [128]:
obj = Class('hi')

In [129]:
dir(obj)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data',
 'prop']

In [130]:
obj.prop

'the prop value'

In [134]:
obj.__dict__

{'prop': 'hi'}

In [133]:
# 이제 coconuts에 왜 weight이랑 price 안 보이는지 알겠음

In [132]:
coconuts.__dict__

{'description': 'Brazilian coconut', '_Quantity#0': 20, '_Quantity#1': 17.95}

In [136]:
# BEGIN LINEITEM_V4B
class Quantity:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self  # <1>
        else:
            return getattr(instance, self.storage_name)  # <2>

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')
# END LINEITEM_V4B

In [137]:
class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [138]:
def quantity():  # <1>
    try:
        quantity.counter += 1  # <2>
    except AttributeError:
        quantity.counter = 0  # <3>

    storage_name = '_{}:{}'.format('quantity', quantity.counter)  # <4>

    def qty_getter(instance):  # <5>
        return getattr(instance, storage_name)

    def qty_setter(instance, value):
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('value must be > 0')

    return property(qty_getter, qty_setter)
# END LINEITEM_V4_PROP

class LineItem:
    weight = quantity()
    price = quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

## 1.3 LineItem 버전 #5: 새로운 디스크립터형

In [139]:
import abc


class AutoStorage:  # <1>
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)  # <2>

In [140]:
class Validated(abc.ABC, AutoStorage):  # <3>

    def __set__(self, instance, value):
        value = self.validate(instance, value)  # <4>
        super().__set__(instance, value)  # <5>

    @abc.abstractmethod
    def validate(self, instance, value):  # <6>
        """return validated value or raise ValueError"""

In [141]:
class Quantity(Validated):  # <7>
    """a number greater than zero"""

    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value

In [142]:
class NonBlank(Validated):
    """a string with at least one non-space character"""

    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value  # <8>

# 2. 오버라이딩 디스크립터와 논오버라이딩 디스크립터

## 2.1 오버라이딩 디스크립터 

In [145]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))

In [163]:
### essential classes for this example ###

class Overriding:  # <1>
    """a.k.a. data descriptor or enforced descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)  # <2>

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class OverridingNoGet:  # <3>
    """an overriding descriptor without ``__get__``"""

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class NonOverriding:  # <4>
    """a.k.a. non-data or shadowable descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)


class Managed:  # <5>
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):  # <6>
        print('-> Managed.spam({})'.format(display(self)))

In [164]:
obj = Managed() # 1

In [165]:
obj.over # 2

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [166]:
Managed.over # 3

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [167]:
obj.over = 7 # 4

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [168]:
obj.over # 5

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [169]:
obj.__dict__['over'] = 8 # 6

In [170]:
vars(obj) # 7

{'over': 8}

In [171]:
obj.over # 8

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [158]:
del obj.over

AttributeError: __delete__

In [161]:
del Managed.over # 디스크립터 지우기

AttributeError: over

In [162]:
obj.over

8

## 2.2 __get__()이 없는 오버라이딩 디스크립터

In [173]:
obj.over_no_get # __get__이 없으니까 그냥 디스크립터 자체 반환

In [174]:
Managed.over_no_get

In [175]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [179]:
obj.over_no_get

In [180]:
obj.__dict__['over_no_get'] = 9

In [184]:
obj.over_no_get # 아니 이게 된다고? 하긴 그냥 클래스 속성이었으니까.. __get__이 없어서 디스크립터가 아닌..? 그런 느낌..?

9

In [182]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [183]:
obj.over_no_get

9

## 2.3 논오버라이딩 디스크립터

In [185]:
obj = Managed()

In [186]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [187]:
obj.non_over = 7

In [189]:
obj.non_over # ???? __get__이 있어서 안 가려질 줄 알았는데.......?

7

In [190]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [191]:
del obj.non_over

In [192]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [193]:
# 아오 개헷갈리네 non_over를 디스크립터고 뭐고 생각 안 하고
# 그냥 클래스 속성으로만 보면 객체 속성이 클래스 속성을 가리는 게 맞긴함
# 근데 쟤는 디스크립터인데..? __set__이 없어서 '논오버라이딩'이라서 
# 디스크립터가 가려지는 ㄱ ㅓ임?

## 4. 클래스 안에서 디스크립터 덮어쓰기

In [194]:
obj = Managed()

In [195]:
Managed.over = 1

In [196]:
Managed.over_no_get = 2

In [197]:
Managed.non_over = 3

In [198]:
obj.over

1

In [199]:
obj.over_no_get

2

In [200]:
obj.non_over

3

# 3. 메서드는 디스크립터

In [201]:
obj = Managed()

In [203]:
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x107319f60>>

In [204]:
Managed.spam

<function __main__.Managed.spam(self)>

In [205]:
obj.spam = 7

In [207]:
obj.spam

7

In [208]:
Managed.spam

<function __main__.Managed.spam(self)>

In [209]:
# 모든 사용자 정의 함수는 __get__() 메서드를 가지고 있다.

In [210]:
dir(Managed.spam)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [211]:
Managed.spam.__get__

<method-wrapper '__get__' of function object at 0x107e9eef0>

In [215]:
Managed.spam.__get__(obj.spam)

<bound method Managed.spam of 7>

In [216]:
obj.spam

7

In [218]:
dir(obj.spam)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'as_integer_ratio',
 'bit_count',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes

In [220]:
import collections


class Text(collections.UserString):

    def __repr__(self):
        return 'Text({!r})'.format(self.data)

    def reverse(self):
        return self[::-1]

In [221]:
word = Text('forward')

In [222]:
word # 1

Text('forward')

In [223]:
word.reverse() # 2

Text('drawrof')

In [225]:
Text.reverse(Text('backward')) # 3

Text('drawkcab')